In [ ]:
#Fundamentos em Big Data
#Tarefa 04 Elastic Search
#Grupo 03
#11/02/2021

In [ ]:
#!pip install elasticsearch
#!pip install csv2es
#!pip install requests

In [ ]:
#Comandos Uteis no Terminal
#pip install -U elasticsearch
#sudo -i service elasticsearch status
#sudo -i service elasticsearch start
#sudo -i service elasticsearch restart

In [1]:
from elasticsearch import helpers, Elasticsearch
import csv
import csv2es

In [2]:
es = Elasticsearch()

In [4]:
df = spark.read.csv('DNPBA2017.csv', header=True)

In [5]:
#df = df.select(['NUMERODN', 'CODESTAB', 'IDADEMAE', 'ESCMAEAGR1']).limit(20).toPandas()
df = df.toPandas()

In [6]:
df.to_csv('DNPBA2017_es.csv', index=False)

### indexng

In [4]:
with open('DNPBA2017_es.csv') as f:
    reader = csv.DictReader(f)
    helpers.bulk(es, reader, index='my-index3', doc_type='my-type')
    
# ler indices no linux com: curl 'localhost:9200/_cat/indices?v'

/home/bigdata/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:193: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [42]:
%%time 
#Gere um subset com todas as mães que tem mais que 40 anos.


content = {
    "query": {
        "range": {
            "IDADEMAE": {
                "gt": 40
            }
        }
    }
}
res = es.search(index="my-index3", body=content)
res

CPU times: user 4.61 ms, sys: 0 ns, total: 4.61 ms
Wall time: 65.3 ms


{'took': 59,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3735, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'my-index3',
    '_type': 'my-type',
    '_id': '1iXZdncB-Jg2IKHd8jQb',
    '_score': 1.0,
    '_source': {'_c0': '2457',
     'NUMERODN': '72268345',
     'CODINST': 'MBA2900700001',
     'ORIGEM': '1',
     'NUMERODV': '8',
     'PREFIXODN': '30',
     'CODESTAB': '2487411',
     'CODMUNNASC': '290070',
     'LOCNASC': '1',
     'IDADEMAE': '49',
     'ESTCIVMAE': '1',
     'ESCMAE': '4',
     'CODOCUPMAE': '612005',
     'QTDFILVIVO': '00',
     'QTDFILMORT': '00',
     'CODMUNRES': '290070',
     'GESTACAO': '5',
     'GRAVIDEZ': '1',
     'PARTO': '1',
     'CONSULTAS': '4',
     'DTNASC': '23012017',
     'HORANASC': '2144',
     'SEXO': '1',
     'APGAR1': '07',
     'APGAR5': '09',
     'RACACOR': '4',
     'PESO': '3070',
     'IDANOMAL': '2',
     'DTCADASTRO': '01022017',
     '

In [46]:
#Obtenha estatísticas descritivas do atributo de escolaridade da mãe (min, max, avg, std)
content = {
    "size": 0,
    "aggs": {
    "dnpba_stats": { "stats": { "field": "ESCMAE" } }
  }
}

 #my_field = field.Text(fields={'raw': field.Keyword()})

In [47]:
res = es.search(index="my-index3",body=content)

RequestError: RequestError(400, 'search_phase_execution_exception', 'Text fields are not optimised for operations that require per-document field data like aggregations and sorting, so these operations are disabled by default. Please use a keyword field instead. Alternatively, set fielddata=true on [ESCMAE] in order to load field data by uninverting the inverted index. Note that this can use significant memory.')

In [ ]:
#Faça uma agregação para saber a quantidade de crianças nascidas em cada município da Bahia.

In [10]:
%%time
#Gere um subset com todas as mães entre 20 e 50 anos que tiveram filhos no primeiro dia do ano.
content = {
    "query": {
        "bool": {
            "must": [
                {
                    "term": {
                        "DTNASC": '01012017'
                    }
                },
                {
                    "range": {
                        "IDADEMAE": {
                                    "gte": 20,
                                    "lte": 50
                            }
                }
                   } 
            ]
        }
    }
}
res = es.search(index="my-index3", body=content)
res

CPU times: user 4.01 ms, sys: 0 ns, total: 4.01 ms
Wall time: 54.9 ms


{'took': 23,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 301, 'relation': 'eq'},
  'max_score': 7.2988896,
  'hits': [{'_index': 'my-index3',
    '_type': 'my-type',
    '_id': 'rCXZdncB-Jg2IKHd-To8',
    '_score': 7.2988896,
    '_source': {'_c0': '3951',
     'NUMERODN': '70510943',
     'CODINST': 'MBA2900800001',
     'ORIGEM': '1',
     'NUMERODV': '9',
     'PREFIXODN': '30',
     'CODESTAB': '2304848',
     'CODMUNNASC': '290080',
     'LOCNASC': '1',
     'IDADEMAE': '30',
     'ESTCIVMAE': '2',
     'ESCMAE': '4',
     'CODOCUPMAE': 'NA',
     'QTDFILVIVO': '00',
     'QTDFILMORT': '00',
     'CODMUNRES': '290080',
     'GESTACAO': '4',
     'GRAVIDEZ': '1',
     'PARTO': '1',
     'CONSULTAS': '3',
     'DTNASC': '01012017',
     'HORANASC': '0235',
     'SEXO': '1',
     'APGAR1': '08',
     'APGAR5': '09',
     'RACACOR': '4',
     'PESO': '2070',
     'IDANOMAL': '2',
     'DTCADASTRO': '06022017',